In [9]:
import pandas as pd
import json
import sqlite3

def bersihkan_data_berdasarkan_analisis(file_input, file_database_output, file_csv_output):
    """
    Fungsi ini membersihkan data dari file JSON sesuai dengan laporan analisis kualitas data
    dan menyimpannya ke dalam database SQLite serta file CSV.
    """
    print(f"Memulai proses pembersihan untuk file: '{file_input}'")

    # --- LANGKAH 1: MEMUAT DATA ---
    with open(file_input, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    df = pd.DataFrame(data_json)
    print(f"Data awal berhasil dimuat dengan {len(df)} baris.")

    # --- LANGKAH 2: MENERAPKAN REKOMENDASI PEMBERSIHAN ---
    print("\n--- Memulai Proses Pembersihan Data ---")

    # Rekomendasi 1: Hapus Baris Error (`judul`)
    jumlah_sebelum = len(df)
    df_bersih = df[~df['judul'].str.contains('HTTPSConnectionPool|NoneType', na=False)].copy()
    print(f"1. Menghapus baris dengan judul error: {jumlah_sebelum - len(df_bersih)} baris dihapus.")

    # Rekomendasi 2: Hapus Baris dengan Negara atau Deskripsi Kosong
    jumlah_sebelum = len(df_bersih)
    df_bersih.dropna(subset=['negara', 'deskripsi'], inplace=True)
    print(f"2. Menghapus baris dengan negara atau deskripsi kosong: {jumlah_sebelum - len(df_bersih)} baris dihapus.")

    # Rekomendasi 3: Isi Nilai Kosong Lainnya
    df_bersih['sutradara'].fillna('Tidak Diketahui', inplace=True)
    df_bersih['tags'].fillna('Tidak Ada', inplace=True)
    # Deskripsi tidak perlu di-fillna lagi karena baris kosongnya sudah dihapus
    print("3. Mengisi nilai kosong untuk sutradara dan tags selesai.")

    # Rekomendasi 4: Validasi Tipe Data
    df_bersih['rating'] = pd.to_numeric(df_bersih['rating'], errors='coerce')
    df_bersih['rilis_date'] = pd.to_datetime(df_bersih['rilis_date'], errors='coerce')
    print("4. Memvalidasi tipe data untuk rating dan rilis_date selesai.")
    
    # Reset index setelah melakukan penghapusan baris
    df_bersih.reset_index(drop=True, inplace=True)

    # Rekomendasi 5: Mengurutkan kolom sesuai prioritas
    urutan_kolom = [
        'judul', 'rilis_date', 'rating', # Kunci Utama
        'negara', 'deskripsi', 'tags', 'sutradara', # Kunci Kontekstual
        'bintang', 'durasi' # Kolom Tambahan
    ]
    # Filter urutan_kolom untuk memastikan semua kolom ada di DataFrame
    kolom_yang_ada = [kolom for kolom in urutan_kolom if kolom in df_bersih.columns]
    df_bersih = df_bersih[kolom_yang_ada]
    print("5. Mengurutkan kolom sesuai prioritas kunci selesai.")

    print("\n--- Proses Pembersihan Selesai ---")
    print(f"Jumlah total baris setelah pembersihan: {len(df_bersih)}")

    # --- LANGKAH 3: MENYIMPAN DATA BERSIH ---
    
    # 3a. Menyimpan ke Database SQLite
    try:
        # Buat salinan DataFrame untuk penyimpanan DB agar tidak mengubah format tanggal di CSV
        df_for_db = df_bersih.copy()
        df_for_db.loc[df_for_db['rilis_date'].notna(), 'rilis_date'] = df_for_db.loc[df_for_db['rilis_date'].notna(), 'rilis_date'].dt.strftime('%Y-%m-%d')
        
        print(f"\nMenyimpan data ke dalam file database '{file_database_output}'...")
        conn = sqlite3.connect(file_database_output)
        df_for_db.to_sql('movies', conn, if_exists='replace', index=False)
        print(f"BERHASIL! Data telah disimpan di tabel 'movies' dalam file '{file_database_output}'.")

    except Exception as e:
        print(f"GAGAL: Terjadi error saat menyimpan ke SQLite. Error: {e}")
    
    finally:
        if 'conn' in locals() and conn:
            conn.close()
            print("Koneksi ke database ditutup.")

    # 3b. Menyimpan ke File CSV
    try:
        print(f"\nMenyimpan data ke dalam file CSV '{file_csv_output}'...")
        df_bersih.to_csv(file_csv_output, index=False, encoding='utf-8')
        print(f"BERHASIL! Data telah disimpan di file '{file_csv_output}'.")
    except Exception as e:
        print(f"GAGAL: Terjadi error saat menyimpan ke CSV. Error: {e}")


# --- MENJALANKAN FUNGSI UTAMA ---
if __name__ == "__main__":
    # Nama file input (data mentah Anda)
    input_file = 'data_film.json'
    
    # Nama file output untuk database bersih Anda
    output_db_file = 'database_film.db'
    
    # Nama file output untuk CSV bersih Anda
    output_csv_file = 'film_bersih.csv'
    
    bersihkan_data_berdasarkan_analisis(input_file, output_db_file, output_csv_file)

Memulai proses pembersihan untuk file: 'data_film.json'
Data awal berhasil dimuat dengan 56276 baris.

--- Memulai Proses Pembersihan Data ---
1. Menghapus baris dengan judul error: 1171 baris dihapus.
2. Menghapus baris dengan negara atau deskripsi kosong: 28323 baris dihapus.
3. Mengisi nilai kosong untuk sutradara dan tags selesai.
4. Memvalidasi tipe data untuk rating dan rilis_date selesai.
5. Mengurutkan kolom sesuai prioritas kunci selesai.

--- Proses Pembersihan Selesai ---
Jumlah total baris setelah pembersihan: 26782

Menyimpan data ke dalam file database 'database_film.db'...


C:\Users\yusri\AppData\Local\Temp\ipykernel_4636\1863328095.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bersih['sutradara'].fillna('Tidak Diketahui', inplace=True)
C:\Users\yusri\AppData\Local\Temp\ipykernel_4636\1863328095.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

BERHASIL! Data telah disimpan di tabel 'movies' dalam file 'database_film.db'.
Koneksi ke database ditutup.

Menyimpan data ke dalam file CSV 'film_bersih.csv'...
BERHASIL! Data telah disimpan di file 'film_bersih.csv'.
